Importing the required libraries

In [539]:
import re
import pickle
import Stemmer
import numpy as np
from collections import defaultdict, OrderedDict
import sys
import os

In the cell below, we have instantiated some global variables, initiated the stemmer and read the index file

In [540]:
STOP_DICT = {}
# INPUT_FILE = '../enwiki-20200801-pages-articles-multistream1.xml-p1p30303'
INDEX_DIR = 'index/'
QUERY = 't:world cup'
if INDEX_DIR[-1]!='/':
    INDEX_DIR+='/'
STOP_DICT = {}
STOP_FILE = ''
if INDEX_DIR.split('/')[0] == '2018114017':
    STOP_FILE = '2018114017/frequent.pickle'
else:
    STOP_FILE = 'frequent.pickle'
with open(STOP_FILE, 'rb') as handle:
    STOP_DICT = pickle.load(handle)
handle.close()
stemmer = Stemmer.Stemmer('english')
# f = open(INDEX_DIR+"index", "r")
# index_string = f.read()
# f.close()
# temp = index_string.split('\n')
# index = defaultdict(list)
# for i in temp:
#     splits = i.split(':')
#     index[splits[0]] = splits[1].split(' ')

In [541]:
len(STOP_DICT.keys())

660

The preprocess function is to process the text. It tokenizes the data, removes unnecessary 
non-ASCII characters and punctuations, stem the words using pystemmer and remove stop words

In [542]:
def preprocess(text):
    text = text.lower()
    tokens = re.sub(r'[^A-Za-z0-9]+', r' ', text).split()
    stemmed_stop_free = []
    for token in tokens:
        if token not in STOP_DICT:
            stemmed_stop_free.append(stemmer.stemWord(token))
    return stemmed_stop_free

The parse_query function parses the function and determines if it is a field query or a normal query

In [543]:
def parse_query(query):
    query_list = query.split(':')
    if len(query_list) == 1:
        return preprocess(query_list[0]) , 0
    else:
        query_dict = {}
        # query_dict[query_list[0]] = ''
        for i in range(1, len(query_list)-1):
            query_dict[query_list[i-1][-1]] = preprocess(query_list[i][:-2])
        query_dict[query_list[-2][-1]] = preprocess(query_list[-1])
        return query_dict, 1

This function is used to handled to handle the normal query

In [544]:
def search(token,f):
    # Compute filesize of open file sent to us
    global indexsize
    indexsize = os.fstat(f.fileno()).st_size
    hi = indexsize
    lo=0
    x=token
    while lo < hi:
        mid = (lo+hi)//2
        f.seek(mid)
        while f.read(1) != '\n':
            pass
        line = f.readline()
        row=line.split(':')
        midval = row[0]
        if midval < x:
            lo=mid
        elif midval > x: 
            hi=mid
        else:
            offsetval = int(row[1])
            ind = open('./index/index','r')
            ind.seek(offsetval)
            line = ind.readline()
            post = line.split(':')[1]
            return post
    return False

In [545]:
# def search(token,delim,f):
#     # Compute filesize of open file sent to us
#     global indexsize
#     indexsize = 0

#     loc = token

#     if indexsize == 0:
#         indexsize = os.fstat(f.fileno()).st_size
#     hi = indexsize

#     lo=0
#     lookfor=loc
#     # print "looking for: ",lookfor
#     while hi-lo > 1:
#         # Find midpoint and seek to it
#         loc = int((hi+lo)/2)
#         # print(" hi = ",hi," lo = ",lo)
#         # print "seek to: ",loc
#         f.seek(loc)
#         # Skip to beginning of line
#         while f.read(1) != '\n':
#             pass

#         line = f.readline()
#         row=line.split(delim)

#         s = row[0]
#         if delim == ' ':
#             s=int(row[0])

#         # post=row[1]
#         # print(s)
#         # print(lookfor>s)

#         if lookfor == s:
#             # print("Found: ",lookfor)
#             if delim == ':':
#                 offsetval = int(row[1])
#                 ind = open('./index/index','r')
#                 ind.seek(offsetval)
#                 line = ind.readline()
            
#                 post = line.split(':')[1]
            
#             elif delim == ' ':
#                 post = ''
#                 for i in range(1,len(row)):
#                     post += row[i]+' '

#             f.close()
#             # print("Found: ",lookfor)
#             return post  # Found

#         if lookfor < s:
#             # print('h1')
#             # Split into lower half
#             hi=loc

#         if lookfor > s:
#             # print('h2')
#             # Split into higher half
#             lo=loc
        
#     # If not found
#     # print("Not Found: ",lookfor)
#     f.close()
#     return False

In [546]:
def search_titles(token,f):
    # Compute filesize of open file sent to us
    global indexsize
    indexsize = os.fstat(f.fileno()).st_size
    hi = indexsize
    lo=0
    x=token
    while lo < hi:
        mid = (lo+hi)//2
        f.seek(mid)
        while f.read(1) != '\n':
            pass
        line = f.readline()
        row=line.split(':')
        midval = int(row[0].strip())
        if midval < x:
            lo=mid
        elif midval > x: 
            hi=mid
        else:
            post = ''
            for i in range(1,len(row)):
                post += row[i]+' '
            f.close()
            return post.strip()
    return False
# def search_titles(token,f):
#     global indexsize
#     indexsize = 0

#     loc = token

#     if indexsize == 0:
#         indexsize = os.fstat(f.fileno()).st_size
#     hi = indexsize

#     lo=0
#     lookfor=loc
#     # print "looking for: ",lookfor
#     while hi-lo > 1:
#         # Find midpoint and seek to it
#         loc = int((hi+lo)/2)
#         # print(" hi = ",hi," lo = ",lo)
#         # print "seek to: ",loc
#         f.seek(loc)
#         # Skip to beginning of line
#         while f.read(1) != '\n':
#             pass

#         line = f.readline()
#         row=line.split(':')
#         s=int(row[0].strip())

#         # post=row[1]
#         # print(s)
#         # print(lookfor>s)

#         if lookfor == s:
#             # print("Found: ",lookfor)
#             post = ''
#             for i in range(1,len(row)):
#                 post += row[i]+' '
            
#             f.close()
#             # print("Found: ",lookfor)
#             return post.strip()  # Found

#         if lookfor < s:
#             # print('h1')
#             # Split into lower half
#             hi=loc

#         if lookfor > s:
#             # print('h2')
#             # Split into higher half
#             lo=loc
        
#     # If not found
#     # print("Not Found: ",lookfor)
#     f.close()
#     return False

In [547]:
def fieldsplit(listing):
    listingdict = {}
    fields = ['t', 'b', 'i', 'c', 'r', 'l','d']
    postinglist = re.findall('[a-z]|[0-9]{1,1000}',listing)
    # print(postinglist)
    for item in range(len(postinglist)):
        cur = postinglist[item]
        if cur in fields:
            listingdict[cur] = int(postinglist[item+1])
    # print(listingdict)
    return listingdict

In [548]:
def find_string_whole(tokens):
    query_listings = {}
    for token in tokens:
        query_listing = search(token,f = open('index/offset','r')).strip().split(' ')
        for i in range(len(query_listing)):
            listing = query_listing[i]
            listingdict = fieldsplit(listing)

            query_listing[i] = listingdict
        
        query_listings[token] = query_listing
    return query_listings
    #     for i in range(len(query_listing)):
    #         listing = query_listing[i]
    #         listingdict = self.fieldsplit(listing)

    #         query_listing[i] = listingdict
        
    #     query_listings[token] = query_listing

    # return self.get_matches(query_listings)

In [549]:
def rank(docs):
    chosen_keys = []
    for key,val in docs[:10]:
        # fp = open("./index/titles",'r')
        # print(key)
        # docs = self.search(key,' ',fp)
        chosen_keys.append(key)
        # fp.close()
    return chosen_keys

In [550]:
def get_matches(query_listings):
    results = defaultdict(lambda: 0)
    docs = defaultdict(lambda: 1)
    weight = {'t':100000, 'b':50, 'i':25, 'c':10, 'r':10, 'l':10}

    for token in query_listings:
        for query_listing in query_listings[token]:
            # print(query_listing)
            for key in query_listing:
                if key == 'd':
                    docs[query_listing['d']] *= 2
                elif key != 'd':
                    if key == 't':
                        # print('weight,100000: ',query_listing['d'])
                        results[query_listing['d']] += query_listing[key]*docs[query_listing['d']]*weight[key]

    sorteddict = [(k, results[k]) for k in sorted(results, key=results.get, reverse=True)]
    # print(sorteddict)
    # print('\n')
    return sorteddict

In [551]:
def print_titles(chosen):
    for i in chosen:
        fp = open("./index/titles",'r')
        docs = search_titles(i,fp)
        print(docs)

In [552]:
def run_whole_query(query):
    docs_intersect = []
    docs_postlist = find_string_whole(query)
    matches = get_matches(docs_postlist)
    rank(matches)
    return docs_intersect, docs_postlist

This function is used to handled to handle the field query

In [553]:
def fieldsplit_parsed(listing,fields):
    listingdict = {}
    # print(fields)

    postinglist = re.findall('[a-z]|[0-9]{1,1000}',listing)

    # print(postinglist)
    # print(listing)

    for item in range(len(postinglist)):
        cur = postinglist[item]
        if cur in fields:
            # print(cur)
            listingdict[cur] = int(postinglist[item+1])
    # print(listingdict)
    return listingdict

In [554]:
def find_string_parsed(query_dict):
    query_listings = {}
    for category in query_dict.keys():
        for token in query_dict[category]:
            query_listing = search(token,f = open('./index/offset','r')).strip().split(' ')
            # print(query_listing)
            for i in range(len(query_listing)):
                listing = query_listing[i]
                listingdict = fieldsplit_parsed(listing,[category,'d'])
                query_listing[i] = listingdict
                # if len(listingdict)!=0:
                #     print(listingdict)
            
            query_listings[token] = query_listing
    return query_listings

In [555]:
def get_matches_parsed(query_listings):
    results = defaultdict(lambda: 0)
    docs = defaultdict(lambda: 1)
    weight = {'t':100000, 'b':50, 'i':25, 'c':10, 'r':10, 'l':10}

    for token in query_listings:
        for query_listing in query_listings[token]:
            # print(query_listing)
            for key in query_listing:
                if key == 'd':
                    docs[query_listing['d']] *= 2
                elif key != 'd':
                    if key == 't':
                        # print('weight,100000: ',query_listing['d'])
                        results[query_listing['d']] += query_listing[key]*docs[query_listing['d']]*weight[key]

    sorteddict = [(k, results[k]) for k in sorted(results, key=results.get, reverse=True)]
    # print(sorteddict)
    # print('\n')
    return sorteddict

In [556]:
def run_parsed_query(query_dict):
    flag = 0
    docs_postlist = find_string_parsed(query_dict)
    # print(docs_postlist)
    matches = get_matches_parsed(docs_postlist)
    chosen = rank(matches)
    print_titles(chosen)
    docs_intersect = []
    # docs_postlist = {}
    # for category in query_dict.keys():
    #     for i in range(len(query_dict[category])):
    #         doc_list = []
    #         post = []
    #         for j in index[query_dict[category][i]]:
    #             if category == 't':
    #                 splits = re.split('t', j)
    #             if category == 'b':
    #                 splits = re.split('b', j)
    #             if category == 'i':
    #                 splits = re.split('i', j)
    #             if category == 'c':
    #                 splits = re.split('c', j)
    #             if category == 'l':
    #                 splits = re.split('l', j)
    #             if category == 'r':
    #                 splits = re.split('r', j)
    #             if len(splits) > 1:
    #                 post.append(j)
    #                 splits2 = re.split('d|b|i|l|r|t|c', j)
    #                 doc_list.append(splits2[1])
    #                 docs_postlist[category+':'+query_dict[category][i]]=j
    #         docs_postlist[category+':'+query_dict[category][i]]=post
    #         if flag == 0:
    #             docs_intersect = doc_list
    #             flag = 1
    #         else:
    #             docs_intersect = np.intersect1d(docs_intersect, doc_list)
    return docs_intersect, docs_postlist

This is used to print the postlist of the queries

In [557]:
def print_postlist(postlist, docs_ids):
    for i in postlist.keys():
        print("Postlist for",i+":")
        for j in postlist[i]:
            print(j,end=' ')
        print()
        print()

In [558]:
parsed, querytype = parse_query(QUERY)
docs_intersect = []
docs_postlist = {}
if querytype == 0:
    docs_intersect, docs_postlist = run_whole_query(parsed)
else:
    docs_intersect, docs_postlist = run_parsed_query(parsed)
# print_postlist(docs_postlist, docs_intersect)

World cup
Antisemitism in the Arab world
Many-worlds interpretation
Hello world
RNA world
Arab world
Fifth world
Fourth world
Second world war
The hello world program
